## 04c MPP bysurface differential analysis MAST full

Celltype:

**MPPs by surface markers (not clustered)**


Run this model:

`zlmCond_all <- zlm(formula = ~condition + Female + n_genes, sca=sca)`

Comparisons:

all cells
- male vs female in all cells controlling for chemical treatment etc
- chemical treatments controlling for sex and n_genes

each cluster
- male vs female in all cells controlling for chemical treatment etc
- chemical treatments controlling for sex and n_genes


done with this docker image:

docker run --rm -d --name test_eva -p 8883:8888 -e JUPYTER_ENABLE_LAB=YES -v /Users/efast/Documents/:/home/jovyan/work r_scanpy:vs5


In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [2]:
import scanpy as sc
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sb
from gprofiler import GProfiler

import rpy2.rinterface_lib.callbacks
import logging

from rpy2.robjects import pandas2ri
import anndata2ri

In [3]:
# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
sc.logging.print_versions()

scanpy==1.4.5.1 anndata==0.7.1 umap==0.3.10 numpy==1.17.3 scipy==1.3.0 pandas==0.25.3 scikit-learn==0.22.2.post1 statsmodels==0.10.0 python-igraph==0.7.1 louvain==0.6.1


In [4]:
%%R
# Load libraries from correct lib Paths for my environment - ignore this!
.libPaths(.libPaths()[c(3,2,1)])

# Load all the R libraries we will be using in the notebook
library(scran)
library(ggplot2)
library(plyr)
library(MAST)

### MPPs

In [5]:
# load data

adata = sc.read(
    './sc_objects/MAST_diffexpr_MPP_annotated.h5ad', cache = True)

In [6]:
#Create new Anndata object for use in MAST with non-batch corrected data as before
adata_raw = adata.copy()
adata_raw.X = adata.raw.X
adata_raw.obs['n_genes'] = (adata_raw.X > 0).sum(1) # recompute number of genes expressed per cell
adata = None

In [7]:
adata_raw

AnnData object with n_obs × n_vars = 5819 × 13099 
    obs: 'assignment', 'batch', 'counts', 'demux_type', 'hto_type', 'rna_type', 'sample', 'n_counts', 'log_counts', 'n_genes', 'percent_mito', 'Female', 'Female_cat', 'Female_str', 'sex_sample', 'rXist', 'leiden'
    var: 'gene_ids', 'feature_types', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'sample_colors', 'sex_sample_colors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'

In [8]:
adata_raw.obs.head()

,assignment,batch,counts,demux_type,hto_type,rna_type,sample,n_counts,log_counts,n_genes,percent_mito,Female,Female_cat,Female_str,sex_sample,rXist,leiden
AAACCCAAGATGGGCT-0-0-0-0-0,MPP,batch1,1021.0,singlet,background,signal,ct,17629.0,9.777698,4802,0.049501,True,True,true,ct_true,3.338417,4
AAAGGATAGTAGTCTC-0-0-0-0-0,MPP,batch1,809.0,singlet,background,signal,ct,13520.0,9.512590,4053,0.056176,False,False,false,ct_false,-0.119305,1
AAAGGATCACGCTGAC-0-0-0-0-0,MPP,batch1,869.0,singlet,background,signal,ct,18169.0,9.807637,4619,0.049802,True,True,true,ct_true,3.066876,2
AAAGGGCAGCAGCGAT-0-0-0-0-0,MPP,batch1,848.0,singlet,background,signal,ct,8510.0,9.049468,2986,0.056965,False,False,false,ct_false,-0.119305,2
AAAGGTACATGAGATA-0-0-0-0-0,MPP,batch1,735.0,singlet,background,signal,ct,5256.0,8.567506,2038,0.057626,True,True,true,ct_true,2.669866,0


#### Select genes expressed in >5% of cells (no adaptive thresholding)

In [9]:
%%R -i adata_raw

#Convert SingleCellExperiment to SingleCellAssay type as required by MAST
sca <- SceToSingleCellAssay(adata_raw, class = "SingleCellAssay")

#Scale Gene detection rate
colData(sca)$n_genes = scale(colData(sca)$n_genes)

# filter genes based on hard cutoff (have to be expressed in at least 5% of all cells)
freq_expressed <- 0.05
expressed_genes <- freq(sca) > freq_expressed
sca <- sca[expressed_genes,]

#rename the sample to condition and make the ct the control
cond<-factor(colData(sca)$sample)
cond<-relevel(cond,"ct")
colData(sca)$condition<-cond

#### run MAST

background:  
`zlmCond_all <- zlm(formula = ~condition + Female + n_genes, sca=sca) # this runs the model`

a formula with the measurement variable (gene expression) on the LHS (left hand side) and 
predictors present in colData on the RHS
expression of genes controlling for cluster, condition, sex + n_genes
questions I can ask:
sex differences controlling for treatments
sex differences controlling for clusters - not necessary analyze all the clusters
overall gene expression changes in treatment


In [10]:
%%R 
#Define & run hurdle model 
zlmCond_all <- zlm(formula = ~condition + Female + n_genes, sca=sca) # this runs the model
summaryCond_all <- summary(zlmCond_all, doLRT=TRUE) # extracts the data, gives datatable with summary of fit, doLRT=TRUE extracts likelihood ratio test p-value
summaryDt_all <- summaryCond_all$datatable # reformats into a table

In [11]:
%%R
head(summaryDt_all)

       primerid component        contrast    Pr..Chisq.        ci.hi
1 0610009B22Rik         C      FemaleTRUE  1.573119e-01  0.004737989
2 0610009B22Rik         C   conditionGCSF  4.940772e-01  0.033506218
3 0610009B22Rik         C conditiondmPGE2  2.041257e-02 -0.004391915
4 0610009B22Rik         C   conditionindo  2.891810e-01  0.011299670
5 0610009B22Rik         C    conditionpIC  2.261133e-05 -0.028569966
6 0610009B22Rik         C         n_genes 1.494891e-249 -0.144555328
        ci.lo         coef           z
1 -0.02949859 -0.012380302  -1.4174865
2 -0.01614560  0.008680311   0.6852961
3 -0.05152801 -0.027959961  -2.3252041
4 -0.03804252 -0.013371423  -1.0622759
5 -0.07732979 -0.052949877  -4.2567774
6 -0.15947825 -0.152016788 -39.9315195


In [12]:
%%R -o female_all -o GCSF_all -o dmPGE2_all -o indo_all -o pIC_all

# reformat for female
result_all_Female <- merge(summaryDt_all[contrast=='FemaleTRUE' & component=='H',.(primerid, `Pr(>Chisq)`)], #P-vals
                  summaryDt_all[contrast=='FemaleTRUE' & component=='logFC', .(primerid, coef)],
                  by='primerid') #logFC coefficients
#Correct for multiple testing (FDR correction) and filtering
result_all_Female[,FDR:=p.adjust(`Pr(>Chisq)`, 'fdr')] # create column named FDR - probably that p.adjust function
female_all = result_all_Female[result_all_Female$FDR<0.01,, drop=F] # create new table where rows with FDR<0.01 are droped
female_all = female_all[order(female_all$FDR),] # sorts the table


# reformat for GCSF
result_all_GCSF <- merge(summaryDt_all[contrast=='conditionGCSF' & component=='H',.(primerid, `Pr(>Chisq)`)], #P-vals
                  summaryDt_all[contrast=='conditionGCSF' & component=='logFC', .(primerid, coef)],
                  by='primerid') #logFC coefficients
#Correct for multiple testing (FDR correction) and filtering
result_all_GCSF[,FDR:=p.adjust(`Pr(>Chisq)`, 'fdr')] # create column named FDR - probably that p.adjust function
GCSF_all = result_all_GCSF[result_all_GCSF$FDR<0.01,, drop=F] # create new table where rows with FDR<0.01 are droped
GCSF_all = GCSF_all[order(GCSF_all$FDR),] # sorts the table


# reformat for dmPGE2
result_all_dmPGE2 <- merge(summaryDt_all[contrast=='conditiondmPGE2' & component=='H',.(primerid, `Pr(>Chisq)`)], #P-vals
                  summaryDt_all[contrast=='conditiondmPGE2' & component=='logFC', .(primerid, coef)],
                  by='primerid') #logFC coefficients
#Correct for multiple testing (FDR correction) and filtering
result_all_dmPGE2[,FDR:=p.adjust(`Pr(>Chisq)`, 'fdr')] # create column named FDR - probably that p.adjust function
dmPGE2_all = result_all_dmPGE2[result_all_dmPGE2$FDR<0.01,, drop=F] # create new table where rows with FDR<0.01 are droped
dmPGE2_all = dmPGE2_all[order(dmPGE2_all$FDR),] # sorts the table


# reformat for indo
result_all_indo <- merge(summaryDt_all[contrast=='conditionindo' & component=='H',.(primerid, `Pr(>Chisq)`)], #P-vals
                  summaryDt_all[contrast=='conditionindo' & component=='logFC', .(primerid, coef)],
                  by='primerid') #logFC coefficients
#Correct for multiple testing (FDR correction) and filtering
result_all_indo[,FDR:=p.adjust(`Pr(>Chisq)`, 'fdr')] # create column named FDR - probably that p.adjust function
indo_all = result_all_indo[result_all_indo$FDR<0.01,, drop=F] # create new table where rows with FDR<0.01 are droped
indo_all = indo_all[order(indo_all$FDR),] # sorts the table

# reformat for pIC
result_all_pIC <- merge(summaryDt_all[contrast=='conditionpIC' & component=='H',.(primerid, `Pr(>Chisq)`)], #P-vals
                  summaryDt_all[contrast=='conditionpIC' & component=='logFC', .(primerid, coef)],
                  by='primerid') #logFC coefficients
#Correct for multiple testing (FDR correction) and filtering
result_all_pIC[,FDR:=p.adjust(`Pr(>Chisq)`, 'fdr')] # create column named FDR - probably that p.adjust function
pIC_all = result_all_pIC[result_all_pIC$FDR<0.01,, drop=F] # create new table where rows with FDR<0.01 are droped
pIC_all = pIC_all[order(pIC_all$FDR),] # sorts the table

In [13]:
%%R -o MAST_raw_all

MAST_raw_all <- summaryDt_all

In [14]:
# save files as .csvs

MAST_raw_all.to_csv('./write/MPP_MAST_raw_surf.csv')
female_all.to_csv('./write/MPP_MAST_female_surf.csv')
GCSF_all.to_csv('./write/MPP_MAST_GCSF_surf.csv')
pIC_all.to_csv('./write/MPP_MAST_pIC_surf.csv')
dmPGE2_all.to_csv('./write/MPP_MAST_dmPGE2_surf.csv')
indo_all.to_csv('./write/MPP_MAST_indo_surf.csv')

In [15]:
# remove python variables

adata_raw = None

In [16]:
%%R
# remove R variables

rm(adata_raw)
rm(zlmCond_all)
rm(summaryDt_all)
rm(summaryCond_all)
rm(MAST_raw_all)

### MPP1s

In [17]:
# load data

adata = sc.read(
    './sc_objects/MAST_diffexpr_MPP1_annotated.h5ad', cache = True)

In [18]:
#Create new Anndata object for use in MAST with non-batch corrected data as before
adata_raw = adata.copy()
adata_raw.X = adata.raw.X
adata_raw.obs['n_genes'] = (adata_raw.X > 0).sum(1) # recompute number of genes expressed per cell
adata = None

In [19]:
adata_raw

AnnData object with n_obs × n_vars = 5343 × 13286 
    obs: 'assignment', 'batch', 'counts', 'demux_type', 'hto_type', 'rna_type', 'sample', 'n_counts', 'log_counts', 'n_genes', 'percent_mito', 'Female', 'Female_cat', 'Female_str', 'sex_sample', 'rXist', 'leiden'
    var: 'gene_ids', 'feature_types', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'sample_colors', 'sex_sample_colors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'

In [20]:
adata_raw.obs.head()

,assignment,batch,counts,demux_type,hto_type,rna_type,sample,n_counts,log_counts,n_genes,percent_mito,Female,Female_cat,Female_str,sex_sample,rXist,leiden
AAACCCATCACATTGG-0-0-0-0-0,MPP1,batch1,781.0,singlet,background,signal,ct,7089.0,8.866864,2545,0.063725,False,False,false,ct_false,0.006942,0
AAACGAAGTACTGGGA-0-0-0-0-0,MPP1,batch1,835.0,singlet,background,signal,ct,10922.0,9.298627,3284,0.055022,True,True,true,ct_true,2.952886,1
AAACGAATCTCGGTAA-0-0-0-0-0,MPP1,batch1,745.0,singlet,background,signal,ct,7894.0,8.974238,2692,0.050905,True,True,true,ct_true,2.404951,0
AAAGAACTCGACCATA-0-0-0-0-0,MPP1,batch1,1210.0,singlet,signal,signal,ct,19285.0,9.867290,4997,0.044844,False,False,false,ct_false,0.006942,1
AAAGGATAGAAGGGAT-0-0-0-0-0,MPP1,batch1,701.0,singlet,background,signal,ct,6741.0,8.816854,2460,0.073811,False,False,false,ct_false,0.006942,0


#### Select genes expressed in >5% of cells (no adaptive thresholding)

In [21]:
%%R -i adata_raw

#Convert SingleCellExperiment to SingleCellAssay type as required by MAST
sca <- SceToSingleCellAssay(adata_raw, class = "SingleCellAssay")

#Scale Gene detection rate
colData(sca)$n_genes = scale(colData(sca)$n_genes)

# filter genes based on hard cutoff (have to be expressed in at least 5% of all cells)
freq_expressed <- 0.05
expressed_genes <- freq(sca) > freq_expressed
sca <- sca[expressed_genes,]

#rename the sample to condition and make the ct the control
cond<-factor(colData(sca)$sample)
cond<-relevel(cond,"ct")
colData(sca)$condition<-cond

#### run MAST

background:  
`zlmCond_all <- zlm(formula = ~condition + Female + n_genes, sca=sca) # this runs the model`

a formula with the measurement variable (gene expression) on the LHS (left hand side) and 
predictors present in colData on the RHS
expression of genes controlling for cluster, condition, sex + n_genes
questions I can ask:
sex differences controlling for treatments
sex differences controlling for clusters - not necessary analyze all the clusters
overall gene expression changes in treatment


In [22]:
%%R 
#Define & run hurdle model 
zlmCond_all <- zlm(formula = ~condition + Female + n_genes, sca=sca) # this runs the model
summaryCond_all <- summary(zlmCond_all, doLRT=TRUE) # extracts the data, gives datatable with summary of fit, doLRT=TRUE extracts likelihood ratio test p-value
summaryDt_all <- summaryCond_all$datatable # reformats into a table

In [23]:
%%R
head(summaryDt_all)

       primerid component        contrast    Pr..Chisq.        ci.hi
1 0610009B22Rik         C      FemaleTRUE  1.630397e-01  0.004565518
2 0610009B22Rik         C   conditionGCSF  8.439253e-01  0.024484451
3 0610009B22Rik         C conditiondmPGE2  4.531509e-01  0.021603937
4 0610009B22Rik         C   conditionindo  1.882372e-01  0.006679173
5 0610009B22Rik         C    conditionpIC  9.056294e-04 -0.016493551
6 0610009B22Rik         C         n_genes 6.408240e-195 -0.120319223
        ci.lo         coef           z
1 -0.02722212 -0.011328300  -1.3969620
2 -0.02995997 -0.002737758  -0.1971151
3 -0.04845231 -0.013424189  -0.7511372
4 -0.03408587 -0.013703349  -1.3177011
5 -0.06378743 -0.040140493  -3.3270231
6 -0.13525281 -0.127786014 -33.5426520


In [24]:
%%R -o female_all -o GCSF_all -o dmPGE2_all -o indo_all -o pIC_all

# reformat for female
result_all_Female <- merge(summaryDt_all[contrast=='FemaleTRUE' & component=='H',.(primerid, `Pr(>Chisq)`)], #P-vals
                  summaryDt_all[contrast=='FemaleTRUE' & component=='logFC', .(primerid, coef)],
                  by='primerid') #logFC coefficients
#Correct for multiple testing (FDR correction) and filtering
result_all_Female[,FDR:=p.adjust(`Pr(>Chisq)`, 'fdr')] # create column named FDR - probably that p.adjust function
female_all = result_all_Female[result_all_Female$FDR<0.01,, drop=F] # create new table where rows with FDR<0.01 are droped
female_all = female_all[order(female_all$FDR),] # sorts the table


# reformat for GCSF
result_all_GCSF <- merge(summaryDt_all[contrast=='conditionGCSF' & component=='H',.(primerid, `Pr(>Chisq)`)], #P-vals
                  summaryDt_all[contrast=='conditionGCSF' & component=='logFC', .(primerid, coef)],
                  by='primerid') #logFC coefficients
#Correct for multiple testing (FDR correction) and filtering
result_all_GCSF[,FDR:=p.adjust(`Pr(>Chisq)`, 'fdr')] # create column named FDR - probably that p.adjust function
GCSF_all = result_all_GCSF[result_all_GCSF$FDR<0.01,, drop=F] # create new table where rows with FDR<0.01 are droped
GCSF_all = GCSF_all[order(GCSF_all$FDR),] # sorts the table


# reformat for dmPGE2
result_all_dmPGE2 <- merge(summaryDt_all[contrast=='conditiondmPGE2' & component=='H',.(primerid, `Pr(>Chisq)`)], #P-vals
                  summaryDt_all[contrast=='conditiondmPGE2' & component=='logFC', .(primerid, coef)],
                  by='primerid') #logFC coefficients
#Correct for multiple testing (FDR correction) and filtering
result_all_dmPGE2[,FDR:=p.adjust(`Pr(>Chisq)`, 'fdr')] # create column named FDR - probably that p.adjust function
dmPGE2_all = result_all_dmPGE2[result_all_dmPGE2$FDR<0.01,, drop=F] # create new table where rows with FDR<0.01 are droped
dmPGE2_all = dmPGE2_all[order(dmPGE2_all$FDR),] # sorts the table


# reformat for indo
result_all_indo <- merge(summaryDt_all[contrast=='conditionindo' & component=='H',.(primerid, `Pr(>Chisq)`)], #P-vals
                  summaryDt_all[contrast=='conditionindo' & component=='logFC', .(primerid, coef)],
                  by='primerid') #logFC coefficients
#Correct for multiple testing (FDR correction) and filtering
result_all_indo[,FDR:=p.adjust(`Pr(>Chisq)`, 'fdr')] # create column named FDR - probably that p.adjust function
indo_all = result_all_indo[result_all_indo$FDR<0.01,, drop=F] # create new table where rows with FDR<0.01 are droped
indo_all = indo_all[order(indo_all$FDR),] # sorts the table

# reformat for pIC
result_all_pIC <- merge(summaryDt_all[contrast=='conditionpIC' & component=='H',.(primerid, `Pr(>Chisq)`)], #P-vals
                  summaryDt_all[contrast=='conditionpIC' & component=='logFC', .(primerid, coef)],
                  by='primerid') #logFC coefficients
#Correct for multiple testing (FDR correction) and filtering
result_all_pIC[,FDR:=p.adjust(`Pr(>Chisq)`, 'fdr')] # create column named FDR - probably that p.adjust function
pIC_all = result_all_pIC[result_all_pIC$FDR<0.01,, drop=F] # create new table where rows with FDR<0.01 are droped
pIC_all = pIC_all[order(pIC_all$FDR),] # sorts the table

In [25]:
%%R -o MAST_raw_all

MAST_raw_all <- summaryDt_all

In [26]:
# save files as .csvs

MAST_raw_all.to_csv('./write/MPP1_MAST_raw_surf.csv')
female_all.to_csv('./write/MPP1_MAST_female_surf.csv')
GCSF_all.to_csv('./write/MPP1_MAST_GCSF_surf.csv')
pIC_all.to_csv('./write/MPP1_MAST_pIC_surf.csv')
dmPGE2_all.to_csv('./write/MPP1_MAST_dmPGE2_surf.csv')
indo_all.to_csv('./write/MPP1_MAST_indo_surf.csv')

In [27]:
# remove python variables

adata_raw = None

In [28]:
%%R
# remove R variables

rm(adata_raw)
rm(zlmCond_all)
rm(summaryDt_all)
rm(summaryCond_all)
rm(MAST_raw_all)

### MPP2s

In [29]:
# load data

adata = sc.read(
    './sc_objects/MAST_diffexpr_MPP2_annotated.h5ad', cache = True)

In [30]:
#Create new Anndata object for use in MAST with non-batch corrected data as before
adata_raw = adata.copy()
adata_raw.X = adata.raw.X
adata_raw.obs['n_genes'] = (adata_raw.X > 0).sum(1) # recompute number of genes expressed per cell
adata = None

In [31]:
adata_raw

AnnData object with n_obs × n_vars = 5884 × 13472 
    obs: 'assignment', 'batch', 'counts', 'demux_type', 'hto_type', 'rna_type', 'sample', 'n_counts', 'log_counts', 'n_genes', 'percent_mito', 'Female', 'Female_cat', 'Female_str', 'sex_sample', 'rXist', 'leiden'
    var: 'gene_ids', 'feature_types', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'sample_colors', 'sex_sample_colors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'

In [32]:
adata_raw.obs.head()

,assignment,batch,counts,demux_type,hto_type,rna_type,sample,n_counts,log_counts,n_genes,percent_mito,Female,Female_cat,Female_str,sex_sample,rXist,leiden
AAAGAACAGGAAGTCC-0-0-0-0-0,MPP2,batch1,1180.0,singlet,signal,signal,ct,22280.0,10.011580,5458,0.051564,True,True,true,ct_true,3.124767,3
AAAGAACGTAACCCTA-0-0-0-0-0,MPP2,batch1,975.0,singlet,background,signal,ct,24253.0,10.096501,5472,0.050004,False,False,false,ct_false,0.178026,4
AAAGGATGTACACGCC-0-0-0-0-0,MPP2,batch1,805.0,singlet,background,signal,ct,19423.0,9.874368,4714,0.036909,True,True,true,ct_true,3.006543,3
AAAGGATGTACGAGTG-0-0-0-0-0,MPP2,batch1,810.0,singlet,background,signal,ct,8478.0,9.046056,2916,0.055863,True,True,true,ct_true,3.358304,0
AAAGGATGTCATCCGG-0-0-0-0-0,MPP2,batch1,467.0,singlet,background,signal,ct,15443.0,9.645170,4524,0.045510,True,True,true,ct_true,3.327009,4


#### Select genes expressed in >5% of cells (no adaptive thresholding)

In [33]:
%%R -i adata_raw

#Convert SingleCellExperiment to SingleCellAssay type as required by MAST
sca <- SceToSingleCellAssay(adata_raw, class = "SingleCellAssay")

#Scale Gene detection rate
colData(sca)$n_genes = scale(colData(sca)$n_genes)

# filter genes based on hard cutoff (have to be expressed in at least 5% of all cells)
freq_expressed <- 0.05
expressed_genes <- freq(sca) > freq_expressed
sca <- sca[expressed_genes,]

#rename the sample to condition and make the ct the control
cond<-factor(colData(sca)$sample)
cond<-relevel(cond,"ct")
colData(sca)$condition<-cond

#### run MAST

background:  
`zlmCond_all <- zlm(formula = ~condition + Female + n_genes, sca=sca) # this runs the model`

a formula with the measurement variable (gene expression) on the LHS (left hand side) and 
predictors present in colData on the RHS
expression of genes controlling for cluster, condition, sex + n_genes
questions I can ask:
sex differences controlling for treatments
sex differences controlling for clusters - not necessary analyze all the clusters
overall gene expression changes in treatment


In [34]:
%%R 
#Define & run hurdle model 
zlmCond_all <- zlm(formula = ~condition + Female + n_genes, sca=sca) # this runs the model
summaryCond_all <- summary(zlmCond_all, doLRT=TRUE) # extracts the data, gives datatable with summary of fit, doLRT=TRUE extracts likelihood ratio test p-value
summaryDt_all <- summaryCond_all$datatable # reformats into a table

In [35]:
%%R
head(summaryDt_all)

       primerid component        contrast    Pr..Chisq.        ci.hi
1 0610009B22Rik         C      FemaleTRUE  6.373351e-03 -0.005485245
2 0610009B22Rik         C   conditionGCSF  7.164278e-01  0.015075893
3 0610009B22Rik         C conditiondmPGE2  1.536900e-07 -0.032350728
4 0610009B22Rik         C   conditionindo  5.259589e-01  0.015972727
5 0610009B22Rik         C    conditionpIC  4.072594e-01  0.013843542
6 0610009B22Rik         C         n_genes 1.316541e-197 -0.121172273
        ci.lo         coef           z
1 -0.03339868 -0.019441961  -2.7302657
2 -0.02193617 -0.003430139  -0.3632842
3 -0.07074231 -0.051546517  -5.2630980
4 -0.03125741 -0.007642340  -0.6342862
5 -0.03413314 -0.010144799  -0.8288794
6 -0.13653095 -0.128851611 -32.8862368


In [36]:
%%R -o female_all -o GCSF_all -o dmPGE2_all -o indo_all -o pIC_all

# reformat for female
result_all_Female <- merge(summaryDt_all[contrast=='FemaleTRUE' & component=='H',.(primerid, `Pr(>Chisq)`)], #P-vals
                  summaryDt_all[contrast=='FemaleTRUE' & component=='logFC', .(primerid, coef)],
                  by='primerid') #logFC coefficients
#Correct for multiple testing (FDR correction) and filtering
result_all_Female[,FDR:=p.adjust(`Pr(>Chisq)`, 'fdr')] # create column named FDR - probably that p.adjust function
female_all = result_all_Female[result_all_Female$FDR<0.01,, drop=F] # create new table where rows with FDR<0.01 are droped
female_all = female_all[order(female_all$FDR),] # sorts the table


# reformat for GCSF
result_all_GCSF <- merge(summaryDt_all[contrast=='conditionGCSF' & component=='H',.(primerid, `Pr(>Chisq)`)], #P-vals
                  summaryDt_all[contrast=='conditionGCSF' & component=='logFC', .(primerid, coef)],
                  by='primerid') #logFC coefficients
#Correct for multiple testing (FDR correction) and filtering
result_all_GCSF[,FDR:=p.adjust(`Pr(>Chisq)`, 'fdr')] # create column named FDR - probably that p.adjust function
GCSF_all = result_all_GCSF[result_all_GCSF$FDR<0.01,, drop=F] # create new table where rows with FDR<0.01 are droped
GCSF_all = GCSF_all[order(GCSF_all$FDR),] # sorts the table


# reformat for dmPGE2
result_all_dmPGE2 <- merge(summaryDt_all[contrast=='conditiondmPGE2' & component=='H',.(primerid, `Pr(>Chisq)`)], #P-vals
                  summaryDt_all[contrast=='conditiondmPGE2' & component=='logFC', .(primerid, coef)],
                  by='primerid') #logFC coefficients
#Correct for multiple testing (FDR correction) and filtering
result_all_dmPGE2[,FDR:=p.adjust(`Pr(>Chisq)`, 'fdr')] # create column named FDR - probably that p.adjust function
dmPGE2_all = result_all_dmPGE2[result_all_dmPGE2$FDR<0.01,, drop=F] # create new table where rows with FDR<0.01 are droped
dmPGE2_all = dmPGE2_all[order(dmPGE2_all$FDR),] # sorts the table


# reformat for indo
result_all_indo <- merge(summaryDt_all[contrast=='conditionindo' & component=='H',.(primerid, `Pr(>Chisq)`)], #P-vals
                  summaryDt_all[contrast=='conditionindo' & component=='logFC', .(primerid, coef)],
                  by='primerid') #logFC coefficients
#Correct for multiple testing (FDR correction) and filtering
result_all_indo[,FDR:=p.adjust(`Pr(>Chisq)`, 'fdr')] # create column named FDR - probably that p.adjust function
indo_all = result_all_indo[result_all_indo$FDR<0.01,, drop=F] # create new table where rows with FDR<0.01 are droped
indo_all = indo_all[order(indo_all$FDR),] # sorts the table

# reformat for pIC
result_all_pIC <- merge(summaryDt_all[contrast=='conditionpIC' & component=='H',.(primerid, `Pr(>Chisq)`)], #P-vals
                  summaryDt_all[contrast=='conditionpIC' & component=='logFC', .(primerid, coef)],
                  by='primerid') #logFC coefficients
#Correct for multiple testing (FDR correction) and filtering
result_all_pIC[,FDR:=p.adjust(`Pr(>Chisq)`, 'fdr')] # create column named FDR - probably that p.adjust function
pIC_all = result_all_pIC[result_all_pIC$FDR<0.01,, drop=F] # create new table where rows with FDR<0.01 are droped
pIC_all = pIC_all[order(pIC_all$FDR),] # sorts the table

In [37]:
%%R -o MAST_raw_all

MAST_raw_all <- summaryDt_all

In [38]:
# save files as .csvs

MAST_raw_all.to_csv('./write/MPP2_MAST_raw_surf.csv')
female_all.to_csv('./write/MPP2_MAST_female_surf.csv')
GCSF_all.to_csv('./write/MPP2_MAST_GCSF_surf.csv')
pIC_all.to_csv('./write/MPP2_MAST_pIC_surf.csv')
dmPGE2_all.to_csv('./write/MPP2_MAST_dmPGE2_surf.csv')
indo_all.to_csv('./write/MPP2_MAST_indo_surf.csv')

In [39]:
# remove python variables

adata_raw = None

In [40]:
%%R
# remove R variables

rm(adata_raw)
rm(zlmCond_all)
rm(summaryDt_all)
rm(summaryCond_all)
rm(MAST_raw_all)

### MPP3s

In [41]:
# load data

adata = sc.read(
    './sc_objects/MAST_diffexpr_MPP3_annotated.h5ad', cache = True)

In [42]:
#Create new Anndata object for use in MAST with non-batch corrected data as before
adata_raw = adata.copy()
adata_raw.X = adata.raw.X
adata_raw.obs['n_genes'] = (adata_raw.X > 0).sum(1) # recompute number of genes expressed per cell
adata = None

In [43]:
adata_raw

AnnData object with n_obs × n_vars = 5197 × 13065 
    obs: 'assignment', 'batch', 'counts', 'demux_type', 'hto_type', 'rna_type', 'sample', 'n_counts', 'log_counts', 'n_genes', 'percent_mito', 'Female', 'Female_cat', 'Female_str', 'sex_sample', 'rXist', 'leiden'
    var: 'gene_ids', 'feature_types', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'sample_colors', 'sex_sample_colors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'

In [44]:
adata_raw.obs.head()

,assignment,batch,counts,demux_type,hto_type,rna_type,sample,n_counts,log_counts,n_genes,percent_mito,Female,Female_cat,Female_str,sex_sample,rXist,leiden
AAACGAAGTTGGACCC-0-0-0-0-0,MPP3,batch1,902.0,singlet,background,signal,ct,13505.0,9.511259,3985,0.057805,True,True,true,ct_true,3.218221,2
AAAGGATGTAGTCTGT-0-0-0-0-0,MPP3,batch1,694.0,singlet,background,signal,ct,8684.0,9.070044,3147,0.059602,True,True,true,ct_true,3.265923,0
AAAGGTATCTTCGACC-0-0-0-0-0,MPP3,batch1,3446.0,singlet,signal,signal,ct,15873.0,9.672815,4295,0.046851,True,True,true,ct_true,3.039035,1
AAAGTGATCTAAGGAA-0-0-0-0-0,MPP3,batch1,783.0,singlet,background,signal,ct,16486.0,9.710449,4116,0.034508,False,False,false,ct_false,0.028160,2
AACAAAGTCGGCTGTG-0-0-0-0-0,MPP3,batch1,951.0,singlet,background,signal,ct,17801.0,9.787459,4633,0.057387,True,True,true,ct_true,3.063702,3


#### Select genes expressed in >5% of cells (no adaptive thresholding)

In [45]:
%%R -i adata_raw

#Convert SingleCellExperiment to SingleCellAssay type as required by MAST
sca <- SceToSingleCellAssay(adata_raw, class = "SingleCellAssay")

#Scale Gene detection rate
colData(sca)$n_genes = scale(colData(sca)$n_genes)

# filter genes based on hard cutoff (have to be expressed in at least 5% of all cells)
freq_expressed <- 0.05
expressed_genes <- freq(sca) > freq_expressed
sca <- sca[expressed_genes,]

#rename the sample to condition and make the ct the control
cond<-factor(colData(sca)$sample)
cond<-relevel(cond,"ct")
colData(sca)$condition<-cond

#### run MAST

background:  
`zlmCond_all <- zlm(formula = ~condition + Female + n_genes, sca=sca) # this runs the model`

a formula with the measurement variable (gene expression) on the LHS (left hand side) and 
predictors present in colData on the RHS
expression of genes controlling for cluster, condition, sex + n_genes
questions I can ask:
sex differences controlling for treatments
sex differences controlling for clusters - not necessary analyze all the clusters
overall gene expression changes in treatment


In [46]:
%%R 
#Define & run hurdle model 
zlmCond_all <- zlm(formula = ~condition + Female + n_genes, sca=sca) # this runs the model
summaryCond_all <- summary(zlmCond_all, doLRT=TRUE) # extracts the data, gives datatable with summary of fit, doLRT=TRUE extracts likelihood ratio test p-value
summaryDt_all <- summaryCond_all$datatable # reformats into a table

In [47]:
%%R
head(summaryDt_all)

       primerid component        contrast    Pr..Chisq.        ci.hi
1 0610009B22Rik         C      FemaleTRUE  6.424882e-01  0.012041779
2 0610009B22Rik         C   conditionGCSF  1.373122e-01  0.006013753
3 0610009B22Rik         C conditiondmPGE2  2.826095e-03 -0.013251214
4 0610009B22Rik         C   conditionindo  6.941992e-01  0.019368688
5 0610009B22Rik         C    conditionpIC  2.122820e-02 -0.004128486
6 0610009B22Rik         C         n_genes 3.607562e-168 -0.123017582
        ci.lo         coef           z
1 -0.01952166 -0.003739942  -0.4644710
2 -0.04383080 -0.018908521  -1.4870240
3 -0.06364322 -0.038447216  -2.9907586
4 -0.02909560 -0.004863459  -0.3933702
5 -0.05082359 -0.027476039  -2.3065392
6 -0.13998017 -0.131498875 -30.3884163


In [48]:
%%R -o female_all -o GCSF_all -o dmPGE2_all -o indo_all -o pIC_all

# reformat for female
result_all_Female <- merge(summaryDt_all[contrast=='FemaleTRUE' & component=='H',.(primerid, `Pr(>Chisq)`)], #P-vals
                  summaryDt_all[contrast=='FemaleTRUE' & component=='logFC', .(primerid, coef)],
                  by='primerid') #logFC coefficients
#Correct for multiple testing (FDR correction) and filtering
result_all_Female[,FDR:=p.adjust(`Pr(>Chisq)`, 'fdr')] # create column named FDR - probably that p.adjust function
female_all = result_all_Female[result_all_Female$FDR<0.01,, drop=F] # create new table where rows with FDR<0.01 are droped
female_all = female_all[order(female_all$FDR),] # sorts the table


# reformat for GCSF
result_all_GCSF <- merge(summaryDt_all[contrast=='conditionGCSF' & component=='H',.(primerid, `Pr(>Chisq)`)], #P-vals
                  summaryDt_all[contrast=='conditionGCSF' & component=='logFC', .(primerid, coef)],
                  by='primerid') #logFC coefficients
#Correct for multiple testing (FDR correction) and filtering
result_all_GCSF[,FDR:=p.adjust(`Pr(>Chisq)`, 'fdr')] # create column named FDR - probably that p.adjust function
GCSF_all = result_all_GCSF[result_all_GCSF$FDR<0.01,, drop=F] # create new table where rows with FDR<0.01 are droped
GCSF_all = GCSF_all[order(GCSF_all$FDR),] # sorts the table


# reformat for dmPGE2
result_all_dmPGE2 <- merge(summaryDt_all[contrast=='conditiondmPGE2' & component=='H',.(primerid, `Pr(>Chisq)`)], #P-vals
                  summaryDt_all[contrast=='conditiondmPGE2' & component=='logFC', .(primerid, coef)],
                  by='primerid') #logFC coefficients
#Correct for multiple testing (FDR correction) and filtering
result_all_dmPGE2[,FDR:=p.adjust(`Pr(>Chisq)`, 'fdr')] # create column named FDR - probably that p.adjust function
dmPGE2_all = result_all_dmPGE2[result_all_dmPGE2$FDR<0.01,, drop=F] # create new table where rows with FDR<0.01 are droped
dmPGE2_all = dmPGE2_all[order(dmPGE2_all$FDR),] # sorts the table


# reformat for indo
result_all_indo <- merge(summaryDt_all[contrast=='conditionindo' & component=='H',.(primerid, `Pr(>Chisq)`)], #P-vals
                  summaryDt_all[contrast=='conditionindo' & component=='logFC', .(primerid, coef)],
                  by='primerid') #logFC coefficients
#Correct for multiple testing (FDR correction) and filtering
result_all_indo[,FDR:=p.adjust(`Pr(>Chisq)`, 'fdr')] # create column named FDR - probably that p.adjust function
indo_all = result_all_indo[result_all_indo$FDR<0.01,, drop=F] # create new table where rows with FDR<0.01 are droped
indo_all = indo_all[order(indo_all$FDR),] # sorts the table

# reformat for pIC
result_all_pIC <- merge(summaryDt_all[contrast=='conditionpIC' & component=='H',.(primerid, `Pr(>Chisq)`)], #P-vals
                  summaryDt_all[contrast=='conditionpIC' & component=='logFC', .(primerid, coef)],
                  by='primerid') #logFC coefficients
#Correct for multiple testing (FDR correction) and filtering
result_all_pIC[,FDR:=p.adjust(`Pr(>Chisq)`, 'fdr')] # create column named FDR - probably that p.adjust function
pIC_all = result_all_pIC[result_all_pIC$FDR<0.01,, drop=F] # create new table where rows with FDR<0.01 are droped
pIC_all = pIC_all[order(pIC_all$FDR),] # sorts the table

In [49]:
%%R -o MAST_raw_all

MAST_raw_all <- summaryDt_all

In [50]:
# save files as .csvs

MAST_raw_all.to_csv('./write/MPP3_MAST_raw_surf.csv')
female_all.to_csv('./write/MPP3_MAST_female_surf.csv')
GCSF_all.to_csv('./write/MPP3_MAST_GCSF_surf.csv')
pIC_all.to_csv('./write/MPP3_MAST_pIC_surf.csv')
dmPGE2_all.to_csv('./write/MPP3_MAST_dmPGE2_surf.csv')
indo_all.to_csv('./write/MPP3_MAST_indo_surf.csv')

In [51]:
# remove python variables

adata_raw = None

In [52]:
%%R
# remove R variables

rm(adata_raw)
rm(zlmCond_all)
rm(summaryDt_all)
rm(summaryCond_all)
rm(MAST_raw_all)

In [53]:
sc.logging.print_versions()
pd.show_versions()

scanpy==1.4.5.1 anndata==0.7.1 umap==0.3.10 numpy==1.17.3 scipy==1.3.0 pandas==0.25.3 scikit-learn==0.22.2.post1 statsmodels==0.10.0 python-igraph==0.7.1 louvain==0.6.1

INSTALLED VERSIONS
------------------
commit           : None
python           : 3.7.3.final.0
python-bits      : 64
OS               : Linux
OS-release       : 4.19.76-linuxkit
machine          : x86_64
processor        : x86_64
byteorder        : little
LC_ALL           : en_US.UTF-8
LANG             : en_US.UTF-8
LOCALE           : en_US.UTF-8

pandas           : 0.25.3
numpy            : 1.17.3
pytz             : 2019.3
dateutil         : 2.8.1
pip              : 19.3.1
setuptools       : 41.6.0.post20191101
Cython           : None
pytest           : 5.3.5
hypothesis       : None
sphinx           : None
blosc            : None
feather          : None
xlsxwriter       : None
lxml.etree       : None
html5lib         : None
pymysql          : None
psycopg2         : None
jinja2           : 2.10.3
IPython          : 7.

In [54]:
%%R

sessionInfo()

R version 3.6.1 (2019-07-05)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Ubuntu 18.04.2 LTS

Matrix products: default
BLAS/LAPACK: /opt/conda/lib/libopenblasp-r0.3.7.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
 [1] parallel  stats4    tools     stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] Matrix_1.2-17               MAST_1.12.0                
 [3] plyr_1.8.4                  ggplot2_3.2.1              
 [5] scran_1.14.1                SingleCellExperiment_1.8.0 
 [7] SummarizedExperiment_1.16.0 DelayedArray_0.12.0        
 [9] BiocParallel_1.20.0         matrixStats_